### Importing necessary packages

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#importing modeling packages

from sklearn.model_selection import train_test_split